In [7]:
import importlib
import sys

sys.path.append("..")  # Ensure the parent directory is in the path

# --- Local Application/Module Imports ---
import data_loader.data_loader
import models.models
import visualizations.visualizations
import utils.utils

importlib.reload(data_loader.data_loader)
from data_loader.data_loader import *

importlib.reload(models.models)
from models.models import *

importlib.reload(visualizations.visualizations)
from visualizations.visualizations import *

importlib.reload(utils.utils)
from utils.utils import *

# --- Notebook Configuration ---
%matplotlib inline
%config InlineBackend.figure_format='retina'

# --- Global Settings ---
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [8]:
train_df, val_df, test_df = load_and_split_data()

print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

Train size: 81677
Validation size: 10210
Test size: 10210


In [9]:
device = get_device()
print(f"Using device: {device}")

Using device: mps


In [10]:
config = load_config_base()

In [11]:
submission_df = load_submission_data()

# Deberta-v3-large Submission

In [12]:
model_name = "microsoft/deberta-v3-large"
model_path = "../fine_tuned_models/deberta-v3-large.bin"

dl_model, loss_fn, submission_data_loader = load_model_with_data_loader(config, model_name, model_path, submission_df)

y_review_texts, y_pred_dl, y_pred_probs_dl, y_test = get_predictions(
    dl_model,
    submission_data_loader,
    device
)

/Users/tnorlha/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}
generate_submission(y_pred_dl, label_map=label_map, output_path="../submissions/deberta_large_submission.csv")

Submission file saved to ../submissions/deberta_large_submission.csv


# Aggregation Submissions

## distilbert-base-multilingual-cased

In [15]:
model_name = "distilbert/distilbert-base-multilingual-cased"
model_path = "../fine_tuned_models/distilbert-base-multilingual-cased.bin"

dbmc_model, loss_fn, submission_data_loader = load_model_with_data_loader(config, model_name, model_path, submission_df)

y_review_texts, y_pred_dbmc, y_pred_probs_dbmc, y_test = get_predictions(
    dbmc_model,
    submission_data_loader,
    device
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## roberta-large

In [16]:
model_name = "FacebookAI/roberta-large"
model_path = "../fine_tuned_models/roberta-large.bin"

rl_model, loss_fn, submission_data_loader = load_model_with_data_loader(config, model_name, model_path, submission_df)

y_review_texts, y_pred_rl, y_pred_probs_rl, y_test = get_predictions(
    rl_model,
    submission_data_loader,
    device
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## deberta-v3-base

In [17]:
model_name = "microsoft/deberta-v3-base"
model_path = "../fine_tuned_models/deberta-v3-base.bin"

db_model, loss_fn, submission_data_loader = load_model_with_data_loader(config, model_name, model_path, submission_df)

y_review_texts, y_pred_db, y_pred_probs_db, y_test = get_predictions(
    db_model,
    submission_data_loader,
    device
)

/Users/tnorlha/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Majority Voting

In [19]:
final_preds_majority = predict_with_ensemble(
    preds_list=[y_pred_dl, y_pred_rl, y_pred_db, y_pred_dbmc],
    strategy="majority"
)

In [20]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}
generate_submission(final_preds_majority, label_map=label_map, output_path="../submissions/majority_voting_submission.csv")

Submission file saved to ../submissions/majority_voting_submission.csv


# Softmax Averaging

In [21]:
final_preds_softmax_avg = predict_with_ensemble(
    preds_list=[y_pred_dl, y_pred_rl, y_pred_db, y_pred_dbmc],
    probs_list=[y_pred_probs_dl, y_pred_probs_rl, y_pred_probs_db, y_pred_probs_dbmc],
    strategy="softmax_avg"
)

In [22]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}
generate_submission(final_preds_softmax_avg, label_map=label_map, output_path="../submissions/softmax_averaging_submission.csv")

Submission file saved to ../submissions/softmax_averaging_submission.csv
